# **Escuela CACIC 2024**
### **CURSO 2: Deep Learning**
#### **Docentes:** Dr. Marcelo Errecalde y Lic. Horacio Thompson


8 de octubre de 2024

Contacto: hjthompson@unsl.edu.ar


---




# Introducción



En esta notebook se explora la implementación de un sistema de Preguntas-Respuestas (*Question-Answering*) utilizando Grandes Modelos de Lenguaje (LLMs). Aunque estos modelos son capaces de generar frases coherentes, uno de los mayores desafíos es que pueden devolver respuestas incorrectas o mal fundamentadas, fenómeno conocido como *alucinaciones*. Para mitigar este problema, surge la técnica conocida como *Retrieval-Augmented Generation* (RAG) que combina el poder generativo de los LLMs con la recuperación de información (*Retrieval Information*), permitiendo la integración de información específica y actualizada, y fundamentalmente mejorando la precisión y justificación de las respuestas.

Para la implementación de esta notebook, la técnica RAG requiere de al menos las siguientes tres etapas:

**1. Base de conocimiento (Knowledge base).** Construiremos una base de datos vectorial que permita indexar y almacenar (de alguna manera) los documentos a analizar, sus embeddings y metadatos. Esto incluye:
- a) Leer documentos en formato PDF
- b) Chunking: dividir los documentos en fragmentos más pequeños
- c) Vectorizar: representar los chunks a través de embeddings, los cuales capturan la semántica y contexto de los textos. Para esto, usaremos un modelo codificador que reciba un texto y devuelva un vector denso de dimensión fija.
- d) Construir la base de datos vectorial: indexar y almacenar los chunks, embeddings y metadatos, facilitando la búsqueda y recuperación de chunks relevantes.

**2. Recuperación de información (Information retrieval).** A partir de una consulta, se recuperan los chunks más relevantes que contengan información útil para devolver una respuesta apropiada. Para ello:
- a) Vectorizar la pregunta: utilizando el mismo modelo codificador que la etapa anterior, de modo que la representación sea coherente con la utilizada para los chunks.
- b) Búsqueda semántica: utilizaremos un recuperador de información (retriever) que busque en la base de datos vectorial los chunks más similares a la consulta. Esto, por supuesto se realizará a nivel de embeddings, por lo que comparan vectores; utilizaremos la métrica Coseno, aunque existen otras. Luego, devolvemos los chunks más relevantes, ordenando esta relevancia de acuerdo a la distancia.

**3. Generación aumentada (Augmented generation).** Crearemos un prompt utilizando los chunks recuperados y la consulta original; luego indicaremos al LLM que genere una respuesta basándose únicamente en el contexto proporcionado por los chunks.

<br>

Para probar el método utilizaremos 3 documentos en formato PDFs:
- Attention is all you need (Vaswani et. al, 2017) [https://arxiv.org/abs/2306.08302]
- Unifying large language models and knowledge graphs: A roadmap (Pan et. al, 2024) [https://arxiv.org/abs/2306.08302]
- Guión de la película "Mentiroso Mentiroso" (Liar Liar). Draft, 1996. [https://imsdb.com/scripts/Liar-Liar.html]


Utilizaremos modelo Gemini (de Google) para realizar consultas y obtener respuestas fundamentadas en el contenido del documento. Finalmente, se comparará este enfoque con el rendimiento del modelo sin aplicar RAG, evidenciando cómo la falta de RI puede llevar a *alucinaciones*.

Nota: En la notebook se explican conceptos clave en las celdas de texto y se ha documentado detalladamente el código experimental para facilitar su comprensión y replicación.


---

# Instalación e importación de librerías

In [ ]:
# Instalación de librerías
!pip install langchain
!pip install google-generativeai langchain-google-genai
!pip install chromadb pypdf2 python-dotenv
!pip install PyPDF
!pip install -U langchain-community
!pip install sentence-transformers
!pip install langchainhub

In [ ]:
# Importar librerías genéricas
import re
from IPython.display import Markdown

# Importar librerías para la preparación de datos
from langchain.document_loaders import PyPDFDirectoryLoader # leer pdfs
from langchain.text_splitter import RecursiveCharacterTextSplitter # dividir en chunks
from langchain.vectorstores import Chroma # BD vectorial
from langchain.embeddings import SentenceTransformerEmbeddings # Embeddings

# Importar librerías para el proceso de Recuperación, Aumentación y Generación
from langchain_google_genai import ChatGoogleGenerativeAI # Conexión a Google Generative con LangChain para conversación
from langchain.prompts import ChatPromptTemplate  # Crear prompts personalizados para enviar a modelos preparados para conversación
from langchain_core.runnables import RunnablePassthrough # Pasar contenido sin modificar (para Pipelines)
from langchain_core.output_parsers import StrOutputParser # Estandarizar salidas de los modelos (para Pipelines)

Para usar Gemini, necesitaremos crear una API Key.

1.   Debes crear tu API Key en: https://aistudio.google.com/
2.   Copiar clave en *Secretos* o asignarla directamente en la variable GOOGLE_API_KEY (consejo: oculta tu API Key 🙂)

In [ ]:
# Trabajar con claves secretas
from google.colab import userdata
GOOGLE_API_KEY = userdata.get('GOOGLE_API_KEY')

# 1. Base de conocimiento

### Cargar PDFs

Cargar los documentos en el entorno actual. Tienes dos opciones:
1. Cargar PDFs desde tu ordenador
2. Cargar tu Drive usando un path

In [ ]:
CARGAR_DESDE_TU_ORDENADOR = True # @param {type:"boolean"}

if CARGAR_DESDE_TU_ORDENADOR:
  # Subir un archivo PDF desde tu computadora
  from google.colab import files
  uploaded = files.upload()
  DATA_PATH = '/content'
else:
  # Montar unidad de Google Drive
  from google.colab import drive
  drive.mount('/content/drive')
  # Path donde están los PDFs
  DATA_PATH = userdata.get('PATH_PDFs')

### Lectura del PDF

In [ ]:
# Leyendo los PDFs del directorio configurado
loader = PyPDFDirectoryLoader(DATA_PATH)
data_on_pdf = loader.load()
# Cantidad de páginas cargadas
len(data_on_pdf)

127

In [ ]:
# Normalizar espacios en el contenido de cada documento
for doc in data_on_pdf:
    doc.page_content = re.sub(r'\s+', ' ', doc.page_content).strip()

### Chunking: partición de los PDFs en *chunks* (fragmentos).

In [ ]:
# Particionando los datos.
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=2000, # Delimitar tamaño de los chunks
    chunk_overlap=100 # overlapping para preservar el contexto
    # separators=["\n\n", "\n", ". ", " ", ""], -> usar otros separadores
)
splits = text_splitter.split_documents(data_on_pdf)
# Cantidad de chunks obtenidos
len(splits)

212

### Vectorizar chunks

Cargar modelo de embeddings (*encoder*).

Puedes buscar tus propios modelos en: https://huggingface.co/sentence-transformers

En este caso, utilizaremos el modelo *paraphrase-MiniLM-L6-v2*, que es una opción adecuada para la comparación semántica de textos y funciona bien en contextos multilingües. También puedes experimentar con el modelo *all-MiniLM-L6-v2*, que está diseñado para propósitos generales.

In [ ]:
# Instanciar el modelo de embedding
model_name = "paraphrase-MiniLM-L6-v2" # "all-MiniLM-L6-v2"  "paraphrase-MiniLM-L6-v2"
embeddings_model = SentenceTransformerEmbeddings(model_name=model_name)

<ipython-input-8-3a5d156374f2>:3: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings_model = SentenceTransformerEmbeddings(model_name=model_name)
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warni

modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/3.73k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

### Base de datos vectorial

Ahora instanciaremos la base de datos vectorial (`vectorstore`), la cual será almacenada en el entorno actual.

Usaremos **Chroma** ([documentación](https://docs.trychroma.com/integrations/langchain)), que es una de las herramientas más utilizadas para almacenar y recuperar información de manera eficiente.

La función *Chroma.from_documents(...)* permite instanciar `vectorstore` pasando como parámetro los chunks obtenidos previamente (`splits`) y el *encoder* (`embedding_model`).

*   Por defecto, esta función utiliza la función *Cosine* para comparar vectores y el algoritmo  *Hierarchical Navigable Small World* (HNSW) para almacenar los vectores; este último permite optimizar la creación de indices, búsqueda y recuperación de documentos.


In [ ]:
# Configura el path a la base de datos
path_db = "/content/db_vectorial"
!mkdir path_db
db_name=path_db.split("/")[-1]

# Almacenamos los chunks en la base de datos
vectorstore = Chroma.from_documents(
    documents=splits,
    embedding=embeddings_model,
    persist_directory=path_db,
    collection_name=db_name,
    )
    # collection_metadata={"hnsw:space": "cosine"} -> por defecto usa cosine (sólo angulo, i.e., dirección)
    # Otras métricas posibles:
    #     collection_metadata={"hnsw:space": "ip"} -> producto interno (inner product)
    #     collection_metadata={"hnsw:space": "l2"} -> distancia euclideana
    # HNSW (Hierarchical navigable small world) -> Sirve para el almacenamiento de vectores,
    #                                              Optimiza cómo se indexan y buscan los vectores.

In [ ]:
# Eliminar BD (igual queda un recurso habilitado, lo mejor es cambiar de nombre de la BD y listo... o reinciciar el entorno)
# del_op = "y" # @param {type:"string"}
# if del_op.lower() == 'y':
#   import shutil
#   print("Eliminando BD")
#   # Elimina el directorio existente si es necesario
#   shutil.rmtree(path_db, ignore_errors=True)
#   del vectorstore

# Listar funciones de vectorstore
# dir(vectorstore)

Inspeccionemos la base de datos vectorial creada.

Observaremos un diccionario formado por los indices (ids), embeddings, metadatos, documentos, entre otros elementos.

In [ ]:
# Ver el formato de vectorstore
vectorstore.get().keys()

dict_keys(['ids', 'embeddings', 'metadatas', 'documents', 'uris', 'data', 'included'])

Podemos ver el contenido. Por defecto, se muestran los indices, metadatos y documentos almacenados.

In [ ]:
# Ver el contenido (por defecto devuelve ids, metadatas, documents)
vectorstore.get()

{'ids': ['f2bcc973-adca-4d8a-9de3-e91c00eb6a0e',
  '272f7840-e978-4776-b45c-3d1704f9c26b',
  '903a3185-d3f0-45e5-ac08-eea88261c9e2',
  '3650e60e-20a1-4d69-bc5b-7cd73b7403de',
  '7dd66b3a-4e0b-4551-a48c-eac35102f123',
  '5c1f0c6e-5c13-4ad9-9497-f0e319481959',
  '70fb7df7-ac23-42f8-8e86-9675178a4f65',
  '1c1634c6-5ca4-4533-9e88-4d4c9b960dad',
  'd803d230-b7ed-46b4-80fa-fc618ca2ce18',
  '76f6151e-55e5-4aa2-82a5-cdefddbeeeaa',
  'e0bef8e7-9bb9-4282-891a-0574367b82bb',
  '6b932455-b420-42f7-a603-93c628088e5d',
  '45ee964c-fd6d-4af8-8436-30c40358266e',
  '9b2b93b6-4f73-4004-a795-685cec056b0c',
  'b48d7681-9f04-4610-82a4-7b408aeef5ca',
  '1eaec0a9-9b26-4cb1-bc7c-005333c1a146',
  'a0bb99a2-68e2-49f8-86a0-55590fd9b152',
  'ca93f828-181a-4688-a980-d5c5c582caa3',
  'f6797a9d-45a3-4f95-a94e-fc15239bc7fe',
  '0655f56b-6699-4591-a18b-e0e0c5f61f9a',
  'd86cda39-8aca-4588-b3a3-d9c185243f3c',
  '6159b42b-d783-44e2-a1d6-62ce4f36bdd7',
  'ce1500ba-edc5-45ad-8778-f0c98f194521',
  'ddef604d-b192-40ca-b576-

Incluso podemos ver los los indices, documentos y *embeddings*

In [ ]:
# Indicamos los elementos a obtener obtener
db_data = vectorstore.get(include=['documents', 'embeddings'])
# Número de elementos a mostrar
N = 5
# Extraer las listas de cada clave
ids = db_data['ids']
docs = db_data['documents']
embs = db_data['embeddings']

# Zipear las listas y mostrar los primeros N elementos
for i, (id_, d, e) in enumerate(zip(ids, docs, embs)):
    if i >= N:
        break
    print(f"ID: {id_}\nDocumento: {d}\nEmbedding: {e}\n")

ID: f2bcc973-adca-4d8a-9de3-e91c00eb6a0e
Documento: JOURNAL OF LATEX CLASS FILES, VOL. ??, NO. ??, MONTH 20YY 1 Unifying Large Language Models and Knowledge Graphs: A Roadmap Shirui Pan, Senior Member, IEEE , Linhao Luo, Yufei Wang, Chen Chen, Jiapu Wang, Xindong Wu, Fellow, IEEE Abstract —Large language models (LLMs), such as ChatGPT and GPT4, are making new waves in the field of natural language processing and artificial intelligence, due to their emergent ability and generalizability. However, LLMs are black-box models, which often fall short of capturing and accessing factual knowledge. In contrast, Knowledge Graphs (KGs), Wikipedia and Huapu for example, are structured knowledge models that explicitly store rich factual knowledge. KGs can enhance LLMs by providing external knowledge for inference and interpretability. Meanwhile, KGs are difficult to construct and evolve by nature, which challenges the existing methods in KGs to generate new facts and represent unseen knowledge. Th

# 2. Recuperación de Información

Crearemos un recuperador de información (`retrieval`).

La función *vectorstore.as_retriever()* permite crear una interfaz sobre la base de datos vectorial que usaremos para realizar búsquedas semánticas y obtener los documentos más relevantes a una consulta.

- Por defecto se aplica la búsqueda por similitud (*search_type="similarity"*) y recupera los K=4 documentos más relevantes (*search_kwargs={"k": N}*).
Estos parámetros se pueden modificar, usando "mmr" (Maximal Marginal Relevance, que permite obtener resultados más diversos)  así como establecer un N mayor a 0 (para recuperar más o menos documentos).

In [ ]:
# Crear el retrieval
retriever = vectorstore.as_retriever()
  # search_type es como va a ser la búsqueda
    # por defecto:
    #   search_type="similarity" -> búsqueda por similitud de vectores (coseno)
    #   search_kwargs={"k": 4} TOP K para la búsqueda de similitud
  # Otros valores:
  #   search_type="mmr" Maximal Marginal Relevance -> resultados sean más diversos
  #   Variar el TOP K para la búsqueda de similitud

# 3. Generación aumentada

### Definción del Sistema QA
Crearemos un Sistema de QA usando *Gemini*.

Usaremos la versión *gemini-pro*, pero también puedes utilizar *gemini-1.5-pro* el cual es más poderoso. Notar que utilizamos nuestra API Key.

Intentaremos que nuestro modelo sea mayormente determinístico. Dos parámetros que ayudan a lograr este comportamiento son:

- temperatura (*temperature*): valor entre 0 y 1. Próximo a 1 es más creativo en sus respuestas, mientras que más cerca de 0 es más conservador.
- *top_p*: probabilidad que controla la diversidad de las palabras generadas. Próximo a 1 habrá más posibles palabras a considerar, mientras que cerca de 0 habrá menos diversidad.


In [ ]:
# Crear el LLM para responder las preguntas
llm = ChatGoogleGenerativeAI(model="gemini-pro", google_api_key=GOOGLE_API_KEY, temperature=0.15, top_p=0.15)

Construimos el *Prompt*, que es una serie de instrucciones que le daremos al LLM para resolver la tarea.

Indicaremos al LLM que debe responder consultas, considerando específicamente el *contexto* formado por los documentos más relaciodados con la consulta.

Esta implementación de la técnica de RAG permite adaptar al modelo para generar respuestas fundamentadas y basadas en información relevante, mejorando así la calidad y precisión de las respuestas obtenidas.

In [ ]:
template = ("Eres un asistente para tareas de question-answering. Usa el siguiente contexto para responder la solictud."
          "Puedes explicar tu respuesta." # Además, debes devolver la información de contexto que usaste para devolver la respuesta."
          "Pregunta {question}\n"
          "Contexto: {context}\n"
          "Respuesta:")

# Definir tu prompt como un template de chat
prompt = ChatPromptTemplate.from_template(template)
prompt

ChatPromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, template='Eres un asistente para tareas de question-answering. Usa el siguiente contexto para responder la solictud.Puedes explicar tu respuesta.Pregunta {question}\nContexto: {context}\nRespuesta:'), additional_kwargs={})])

Implementamos un Pipeline: secuencia ordenada de operaciones, donde el resultado de una se utiliza en la siguiente operación.

1. Crear el contexto usando:
-   `retriever` el cual pasa a: `format_docs` para unir los documentos más relevantes en una única pieza de texto.
-   El contenido de `question` pasa sin modificaciones (`RunnablePassthrough`).

2. Crear un diccionario (`context` y `question`) el cual pasa a: `prompt`.
3. Construir el `prompt` el cual pasa a: `LLM` (Aquí sucede el RAG).
4. `LLM` genera la respuesta y pasa a: `StrOutputParser` el cual parsea el resultado en formato estándar.

In [ ]:
def format_docs(docs):
    # Funcion auxiliar para enviar el contexto al modelo como parte del prompt
    return "\n".join(doc.page_content for doc in docs)

rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

# **Ahora sí, llegó el momento de hacer preguntas al LLM usando RAG**

In [ ]:
# Preguntas a la Base de Conocimiento
pregunta = "Que ventajas y desventajas tienen los kgs?" # @param {type:"string"}
response = rag_chain.invoke(pregunta)

print("Respuesta del modelo:")
Markdown(response)

Respuesta del modelo:


**Ventajas de los KGs:**

* **Conocimiento estructural:** Los KGs almacenan hechos en un formato estructurado (es decir, triples), que pueden ser entendidos tanto por humanos como por máquinas.
* **Precisión:** Los hechos en los KGs suelen ser curados o validados manualmente por expertos, lo que los hace más precisos y fiables que los de los LLM.
* **Decisión:** El conocimiento fáctico en los KGs se almacena de forma decisiva. El algoritmo de razonamiento en los KGs también es determinista, lo que puede proporcionar resultados decisivos.
* **Interpretabilidad:** Los KGs son conocidos por su capacidad de razonamiento simbólico, que proporciona un proceso de razonamiento interpretable que puede ser entendido por los humanos.
* **Conocimiento específico del dominio:** Muchos dominios pueden construir sus KGs por expertos para proporcionar un conocimiento preciso y fiable específico del dominio.
* **Conocimiento en evolución:** Los hechos en los KGs están en continua evolución. Los KGs pueden actualizarse con nuevos hechos insertando nuevos triples y eliminando los obsoletos.

**Desventajas de los KGs:**

* **Incompletez:** Los KGs son difíciles de construir y a menudo están incompletos, lo que limita la capacidad de los KGs para proporcionar un conocimiento completo.
* **Falta de comprensión del lenguaje:** La mayoría de los estudios sobre KGs modelan la estructura del conocimiento, pero ignoran la información textual en los KGs. La información textual en los KGs suele ignorarse en las tareas relacionadas con los KGs, como la finalización de los KGs y el KGQA.
* **Hechos no vistos:** Los KGs cambian dinámicamente, lo que dificulta el modelado de entidades no vistas y la representación de nuevos hechos.

Verifiquemos los documentos más relevantes (chunks) que usó el modelo para generar la respuesta.

- Puedes buscar estos fragmentos en el documento original

In [ ]:
# Buscar los documentos más relevantes para una pregunta con el retriever
results = retriever.get_relevant_documents(pregunta)
# Almacenar los splits usados
splits_usados = [doc.page_content for doc in results]
# display(splits_usados)
print("Chunk utilizados:")
for i, split in enumerate(splits_usados):
    print(f"Chunk {i+1}:")
    print(split)
    print(len(split))

Chunk utilizados:
Chunk 1:
they can mutually validate each other, resulting in efficient and effective solutions powered by dual-driving wheels. There- fore, we can anticipate increasing attention to unlock the po- tential of integrating KGs and LLMs for diverse downstream applications with both generative and reasoning capabilities in the near future. 8 C ONCLUSION Unifying large language models (LLMs) and knowledge graphs (KGs) is an active research direction that has at- tracted increasing attention from both academia and in- dustry. In this article, we provide a thorough overview of the recent research in this field. We first introduce different manners that integrate KGs to enhance LLMs. Then, we introduce existing methods that apply LLMs for KGs and establish taxonomy based on varieties of KG tasks. Finally, we discuss the challenges and future directions in this field.We envision that there will be multiple stages (milestones) in the roadmap of unifying KGs and LLMs, as shown in

<ipython-input-19-fcc82591d534>:2: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  results = retriever.get_relevant_documents(pregunta)


# **Hacer preguntas al LLM sin usar RAG**
Examinemos qué respondería el modelo sin un contexto, es decir, usando sólo el conocimiento aprendido durante su entrenamiento.

Puedes evaluar si el modeo genera *alucinaciones* según tu criterio e interpretación.

In [ ]:
noRAG_template = (
    "Eres un asistente para tareas de question-answering."
    "ebes responder a la pregunta según tu conocimiento.\n"
    "Pregunta: {question}.\n"
    "Respuesta:"
)
noRAG_prompt = ChatPromptTemplate.from_template(noRAG_template)

In [ ]:
noRAG_chain = (
    {"question": RunnablePassthrough()}
    | noRAG_prompt
    | llm
    | StrOutputParser()
)

response = noRAG_chain.invoke(pregunta)

print("Respuesta del modelo sin RAG:")
Markdown(response)

Respuesta del modelo sin RAG:


**Ventajas de los kilogramos (kg):**

* **Estándar internacional:** El kilogramo es la unidad base de masa en el Sistema Internacional de Unidades (SI), lo que lo convierte en un estándar reconocido internacionalmente.
* **Precisión:** El kilogramo se define con una precisión extremadamente alta, lo que lo hace adecuado para mediciones científicas y técnicas.
* **Fácil de usar:** Los kilogramos son una unidad familiar y fácil de entender, lo que los hace convenientes para el uso diario.
* **Ampliamente utilizado:** Los kilogramos se utilizan en una amplia gama de aplicaciones, desde el comercio hasta la medicina y la ingeniería.

**Desventajas de los kilogramos:**

* **Dependencia de un artefacto físico:** El kilogramo se define en relación con un artefacto físico específico, el Prototipo Internacional del Kilogramo (IPK), que se almacena en Francia. Esto introduce una posible fuente de error si el IPK se daña o se pierde.
* **Posible deriva:** Se ha observado que el IPK ha perdido masa con el tiempo, lo que plantea preocupaciones sobre la estabilidad de la definición del kilogramo.
* **Limitaciones en la medición:** Los kilogramos son difíciles de medir con precisión en escalas pequeñas, lo que puede ser un inconveniente para ciertas aplicaciones.
* **Confusión con otras unidades:** Existen otras unidades de masa, como libras y onzas, que pueden causar confusión cuando se convierten a kilogramos.

Como quizás hayas podido ver, el modelo devuelve una respuesta presumiblemente es errónea, incompleta o fuera de contexto.

Considerar que *Gemini* fue entrenado con datos de Google e incluso está conectado al motor de búsqueda en tiempo real, por lo que es probable que tienda a generar respuestas basadas en información disponible en la Web.



---
# **Conclusión:**


En esta notebook desarrollamos un sistema sencillo de Question-Answering para analizar documentos en formato PDF, utilizando LLMs e implementando la técnica RAG, con el objetivo de mitigar el fenómeno de las alucinaciones. Esto implicó la aplicación y comprensión de diversos conceptos, tales como recuperación de información, bases de datos vectoriales, embeddings, búsqueda semántica, métricas de similitud y prompting. Aplicando el modelo Gemini, observamos que es posible obtener respuestas justificadas considerando el contexto del documento en estudio. Además, mediante un ejemplo sencillo, confirmamos que sin RAG el modelo puede generar información coherente pero incorrecta, lo que podría confundir a la audiencia. Esto resalta la importancia de la técnica RAG, posicionándola como un enfoque interesante para abordar los diversos desafíos que implica el uso de los LLMs.

<br>

**Esperamos que te haya gustado y hayas aprendido algo nuevo! 🙂**


# Bonus: obtener requirements.txt

### Ejecuta la siguiente celda y guarda todas las dependencias.
#### Esto es útil porque las librerías de conda y pip están en constante actualización, pudiendo quedar tu implementación obsoleta.

*    Debes indicar en la lista (LIBRARIES_TO_SAVE) todas las librerias que utilices en esta notebook ("import ...", "from ...", etc.)
*   Se creará un archivo *requirements.txt* que luego podrás utilizar para ejecutar esta notebook en cualquier momento

In [ ]:
import os
import subprocess
from google.colab import files
"""
!pip install langchain
!pip install google-generativeai langchain-google-genai
!pip install chromadb pypdf2 python-dotenv
!pip install PyPDF
!pip install -U langchain-community
!pip install sentence-transformers
!pip install langchainhub
"""

# Agregar cada librería (ya deben estar importadas/instaladas)
LIBRARIES_TO_SAVE = ["langchain", "google-generativeai", "langchain-google-genai",
                     "chromadb", "pypdf2", "python-dotenv", "PyPDF", "langchain-community", "sentence-transformers", "langchainhub"]

# Obtener versiones de las librerías
out = subprocess.run(["pip", "freeze"], capture_output=True, text=True).stdout
filter = [ line
          for line in out.split('\n')
              if any(library.lower() == line.split("==")[0].lower()
                          for library in LIBRARIES_TO_SAVE  )
        ]
# Guardar requirements.txt (en entorno actual)
with open("requirements.txt", 'w') as f:
  f.write("\n".join(filter))

# Chequear requirements.txt
print("Check versions of libraries:")
with open("requirements.txt", 'r') as f:
  print(f"{f.read()}")

# req_file_path = "/content"
# # Puedes guardar requirements.txt en tu directorio raíz
# if input(f"Save in: {req_file_path}? (y/n)").lower() == 'y':
#   output_path = os.path.join(req_file_path, "requirements.txt")
#   with open(output_path, 'w') as f:
#     f.write("\n".join(filter))
#   print("> Saved!", output_path)


# Puedes descargar requirements.txt
if input("Download file? (y/n): ").lower() == 'y':
  files.download('requirements.txt')

# Para instalar las librerías en tu nueva notebook puedes usar:
# print("Instalando librerias...")
# !pip install -r requirements.txt
# print("Reiniciar automáticamente la notebook para luego cargar las librerías instaladas")
# os._exit(00)

Check versions of libraries:
chromadb==0.5.11
google-generativeai==0.7.2
langchain==0.3.1
langchain-community==0.3.1
langchain-google-genai==2.0.0
langchainhub==0.1.21
pypdf==5.0.1
PyPDF2==3.0.1
python-dotenv==1.0.1
sentence-transformers==3.1.1
Download file? (y/n): y


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>